In [1]:
import os
import glob
import netCDF4
import logging
import numpy as np
import cmocean
import emodnetchemistry
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import calendar
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
plt.rcParams.update({'font.size': 16})
plt.rc('figure', facecolor='w')

## Paths

In [4]:
figdir = "/home/ctroupin/dox/EMODnet-Chemistry/WOA-DIVA/"
woadataurl = "https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/oxygen/all/1.00/woa18_all_o00_01.nc"
divafile = "/data/EMODnet/Eutrophication/Products/Results-0.1/Water body dissolved oxygen concentration_background_test_it2000_800km.nc"
divafile = "/data/EMODnet/Eutrophication/Products/Water body dissolved oxygen concentration-res-0.1-epsilon2-1.0-lenx-800000.0-maxit-2000/Water body dissolved oxygen concentration_background.nc"
os.path.isfile(divafile) & os.path.isdir(figdir)

True

## Grid parameters

In [5]:
deltalon = 0.1
deltalat = 0.1
domain = [-45., 70., 24., 83.]
longrid = np.arange(domain[0], domain[1] + .0001, deltalon)
latgrid = np.arange(domain[2], domain[3] + .0001, deltalat)
if not os.path.exists(figdir):
    os.makedirs(figdir)

In [24]:
def read_oxy_woa(datafile, domain, filetype="woa"):
    with netCDF4.Dataset(datafile, "r") as nc:
        lon = nc.variables["lon"][:]
        lat = nc.variables["lat"][:]
        goodlon = np.where( (lon >= domain[0]) & (lon <= domain[1]) )[0]
        goodlat = np.where( (lat >= domain[2]) & (lat <= domain[3]) )[0]
        print(goodlon, goodlat)
        lon = lon[goodlon]
        lat = lat[goodlat]
        depth = nc.variables["depth"][:]
        
        if filetype == "woa":
            oxy = nc.variables["o_an"][0,:,goodlat,goodlon]
        elif filetype == "diva":
            oxy = nc.variables["Water body dissolved oxygen concentration"][0,:,goodlat,goodlon]
        
    return lon, lat, depth, oxy, goodlon, goodlat

In [13]:
def plot_oxy_map(llon, llat, field, depth, figname=None):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(111)
    pcm = m.pcolormesh(llon, llat, field, latlon=True, vmin=200., vmax=375.)
    cb = plt.colorbar(pcm, shrink=.6, extend="both")
    cb.set_label(r"$\mu$moles/kg", rotation=0, ha="left")
    cb.set_ticks(np.arange(200., 376., 25))
    m.drawmeridians(np.arange(-40., 60., 20.), zorder=2, labels=[0, 0, 0, 1], fontsize=14,  
                    linewidth=.25)
    m.drawparallels(np.arange(20., 80., 10.), zorder=2, labels=[1, 0, 0, 0], fontsize=14, 
                    linewidth=.25)
    m.fillcontinents(color=".85", zorder=3)
    m.drawcoastlines(linewidth=0.1, zorder=4)
    plt.title("Oxygen concentration at {} m".format(depth))
    if figname is not None:
        plt.savefig(figname, dpi=300, bbox_inches="tight")
    # plt.show()
    plt.close()

In [14]:
m = Basemap(projection='merc', llcrnrlon=domain[0], llcrnrlat=domain[2],
            urcrnrlon=domain[1], urcrnrlat=domain[3],
            lat_ts=0.5 * (domain[2] + domain[3]), resolution='i')

In [25]:
lon1, lat1, depth1, oxy1, glon1, glat1 = read_oxy_woa(woadataurl, domain, filetype='woa')
lon2, lat2, depth2, oxy2, glon2, glat2 = read_oxy_woa(divafile, domain, filetype='diva')
llon1, llat1 = np.meshgrid(lon1, lat1)
llon2, llat2 = np.meshgrid(lon2, lat2)

[135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249] [114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170 171 172]
[   0    1    2 ... 1148 1149 1150] [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62

## Loop on the depths

In [10]:
def plot_WOA_DIVAnd_comparison(lon1, lat1, field1, lon2, lat2, field2, depth, figname=None,
                              vmin=200., vmax=375.):

    fig = plt.figure(figsize=(12, 10))
    ax1 = plt.subplot(121)
    pcm = m.pcolormesh(lon1, lat1, field1, latlon=True, vmin=vmin, vmax=vmax)
    m.drawmeridians(np.arange(-40., 70., 20.), zorder=2, labels=[0, 0, 0, 1], fontsize=14,  
                    linewidth=.25)
    m.drawparallels(np.arange(20., 81., 10.), zorder=2, labels=[1, 0, 0, 0], fontsize=14, 
                    linewidth=.25)
    m.fillcontinents(color=".85", zorder=3)
    m.drawcoastlines(linewidth=0.1, zorder=4)
    plt.title("World Ocean Atlas 2018 at {} m".format(int(depth)))

    ax2 = plt.subplot(122)
    pcm = m.pcolormesh(lon2, lat2, field2, latlon=True, vmin=vmin, vmax=vmax)
    m.drawmeridians(np.arange(-40., 70., 20.), zorder=2, labels=[0, 0, 0, 1], fontsize=14,  
                    linewidth=.25)
    m.drawparallels(np.arange(20., 81., 10.), zorder=2, labels=[1, 0, 0, 0], fontsize=14, 
                    linewidth=.25)
    m.fillcontinents(color=".85", zorder=3)
    m.drawcoastlines(linewidth=0.1, zorder=4)
    plt.title("DIVAnd")


    cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.05])
    cb = plt.colorbar(pcm,  cax=cbar_ax, extend="both", orientation="horizontal")
    cb.set_label(r"$\mu$moles/kg", rotation=0, ha="left")
    cb.set_ticks(np.arange(200., 376., 25))
    plt.savefig(figname, dpi=300, bbox_inches="tight")
    plt.close()

In [28]:
for index2, d2 in enumerate(depth2[0:1]):
    print("Working on depth {}".format(d2))
    index1 = np.where(depth1 == d2)[0][0]

    
    figname = os.path.join(figdir, "oxygen_DIVAnd_WOA_year_{}.jpg".format(str(int(d2)).zfill(4)))
    plot_WOA_DIVAnd_comparison(llon1, llat1, oxy1[index1,:,:], 
                               llon2, llat2, oxy2[index2,:,:], 
                               d2, figname=figname, vmin=200, vmax=400.)

Working on depth 0.0


In [38]:
depth = 0.0
figname = os.path.join(figdir, "oxygen_DIVAnd_WOA_year_{}.jpg".format(str(int(depth)).zfill(4)))
plot_WOA_DIVAnd_comparison(llon1, llat1, oxy1[0,:,:], llon2, llat2, oxy2[0,:,:], depth, figname=figname)

In [22]:
oxy1.shape

(1, 102, 59, 115)

In [23]:
oxy2.shape

(1, 33, 591, 1151)